In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from timeit import default_timer as timer
import numpy as np
import pandas as pd
from queue import Queue

from IPython.core.debugger import set_trace

In [16]:
test_schematic = '''COM)B
B)C
C)D
D)E
E)F
B)G
G)H
D)I
E)J
J)K
K)L'''

In [65]:
with open('input.txt') as inp:
    in_schematic = inp.read()

In [ ]:
# tree data structure with stars as nodes, direct orbits as children

In [75]:
class star:
    def __init__(self, name = None):
        self.direct_orbits = []
        
        self.n_direct_orbits = 0
        self.n_indirect_orbits = 0
        self.name = name
        self.updated = False
        
        self.min_d_to_a = pd.np.inf
        self.min_d_to_b = pd.np.inf
    
    def add_orbiter(self, other):
        self.direct_orbits.append(other)
        self.n_direct_orbits += 1

    def update_indirect_orbits(self):
        if self.updated:
            return self.n_indirect_orbits
        else:
            self.updated = True
            
        if not self.direct_orbits:
            self.n_indirect_orbits = 0
        else:
            for orbiter in self.direct_orbits:
                self.n_indirect_orbits += orbiter.n_direct_orbits + orbiter.update_indirect_orbits()
        return self.n_indirect_orbits
        
    def find_min_distances(self, a, b):
        if a in self.direct_orbits:
            self.min_d_to_a = 1
        elif a is self:
            self.min_d_to_a = 0
        
        if b in self.direct_orbits:
            self.min_d_to_b = 1
        elif b is self:
            self.min_d_to_b = 0
        
        
    
    def __repr__(self):
        return '{}'.format([n.name for n in self.direct_orbits])
        

In [67]:
def star_map_from_schematic(schematic):
    relationships = schematic.split('\n')
    star_map = {}
    
    for orbit in relationships:
        stars = orbit.split(')')
        orbiter = stars[1]
        orbited = stars[0]

        if orbiter not in star_map:
            star_map[orbiter] = star(orbiter)
        
        if orbited in star_map:
            star_map[orbited].add_orbiter( star_map[orbiter] )
        else:
            star_map[orbited] = star(orbited)
            star_map[orbited].add_orbiter( star_map[orbiter] )
            
    return star_map

In [71]:
star_map = star_map_from_schematic(in_schematic)

In [72]:
q = Queue()
q.put(star_map['COM'])
total_orbits = 0
while not q.empty():
    node = q.get_nowait()
    node.update_indirect_orbits()
    total_orbits += node.n_direct_orbits + node.n_indirect_orbits
    for n in node.direct_orbits:
        q.put(n)

In [73]:
total_orbits

241064